In [3]:
# Import libraries

import os
import cv2
import json
import tensorflow as tf
import numpy as np
import pickle

In [ ]:
class Data_Loader():
    def __init__(self, data_file_path, label_file_path, n_images):
        
        self.data_file_path = data_file_path
        self.label_file_path = label_file_path
        self.n_images = n_images
    
    
    def load_image_data(self): 
        X = []
        Y = [] 
        for file in os.listdir(self.data_file_path):
            file_path = os.path.join(self.data_file_path, file)
            if file.endswith('.jpg'):
                img_annotations = self.__load_annotations(file)
                if img_annotations is not None:
                    Y.append(img_annotations)
                    img = cv2.imread(file_path)
                    X.append(img)
                if len(X) >= self.n_images :
                    return X, Y
        return X, Y
    
    
    def __load_annotations(self, image_file_name):
        file_name = image_file_name.split('.jpg')[0]
        json_file_name = file_name + '.json'
        json_file_path = os.path.join(self.label_file_path, json_file_name)
        if os.path.isfile(json_file_path):
            f = open(json_file_path)
            return json.load(f)
        else:
            return None
        

In [ ]:
class Image_Processor():
    
    def __init__(self, images):
        self.images = images
        self.min_width, self.min_height = self.__find_smallest_image_width_and_height()
        self.resized_images = self.__resize_images()
    
    
    def __resize_images(self):
        
        X_resized = []
        
        for img in self.images:
            X_resized.append(tf.image.resize(img, 
                                             size=(self.min_width, self.min_height)))
            
        return np.array(X_resized)
    
    
    def __find_smallest_image_width_and_height(self):
        
        min_width = np.size(self.images[0], 0)
        min_height = np.size(self.images[0], 1)
        
        for img in self.images[1:]:
            
            if np.size(img, 0) < min_width:
                min_width = np.size(img, 0)
                
            if np.size(img, 1) < min_height:
                min_height = np.size(img, 1)
                
        return min_width, min_height

In [ ]:
def build_model(input_shape):
    
    model = tf.keras.Sequential()

    # add first convolution layer to the model
    model.add(tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding='same',
        data_format='channels_last',
        name='conv_1',
        activation='relu'))
    
    
    # add a max pooling layer with pool size (2,2) and strides of 2
    # (this will reduce the spatial dimensions by half)
    model.add(tf.keras.layers.MaxPool2D(
        pool_size=(2, 2),
        name='pool_1'))
    
    
    # add second convolutional layer
    # model.add(tf.keras.layers.Conv2D(
    #     filters=64,
    #     kernel_size=(5, 5),
    #     strides=(1, 1),
    #     padding='same',
    #     name='conv_2',
    #     activation='relu'))
    
    # # add second max pooling layer with pool size (2,2) and strides of 2
    # # (this will further reduce the spatial dimensions by half)
    # model.add(tf.keras.layers.MaxPool2D(
    #     pool_size=(2, 2), name='pool_2')
    # )
    
    
    # add a fully connected layer (need to flatten the output of the previous layers first)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=16,
        name='fc_1',
        activation='relu'))
    
    # add dropout layer
    # model.add(tf.keras.layers.Dropout(
    #     rate=0.5))
    
    # add the last fully connected layer
    # this last layer sets the activation function to "None" in order to output the logits
    # note that passing activation = "sigmoid" will return class memembership probabilities but
    # in TensorFlow logits are prefered for numerical stability
    # set units=1 to get a single output unit (remember it's a binary classification problem)
    model.add(tf.keras.layers.Dense(
        units=5,
        name='fc_2',
        activation=None))
    
    model.build(input_shape=input_shape)
    
    return model

In [ ]:
import os
os.getcwd()

In [ ]:
data_file_path = './benetech-making-graphs-accessible/train/images'
label_file_path = './benetech-making-graphs-accessible/train/annotations'

In [ ]:
n_images = 700

# Load in raw data
X_raw, Y_raw = Data_Loader(data_file_path, 
                           label_file_path, n_images).load_image_data()

In [ ]:
X_processed = Image_Processor(X_raw)
X_resized = X_processed.resized_images
X_gray = X_resized / 255.0

In [ ]:
Y_plot_type = [y['chart-type'] for y in Y_raw]

plot_type_mapping = {'scatter':0, 'line':1, 'dot':2, 'vertical_bar':3, 
                       'horizontal_bar':4}
Y_plot_type = [plot_type_mapping[plot_type] for plot_type in Y_plot_type]
Y_plot_type = np.array(Y_plot_type)
shuffle = tf.random.shuffle(tf.range(tf.shape(X_gray)[0], dtype=tf.int32))
X_all = tf.gather(X_gray, shuffle)
y_all = tf.gather(Y_plot_type, shuffle)

split = int(len(X_raw) * 0.8)

X_train, Y_train = X_all[:split], y_all[:split]
X_val, Y_val = X_all[split:], y_all[split:]

model = build_model(
     input_shape=(None, X_processed.min_width, X_processed.min_height, 3))

model.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                     epochs=25,
                     validation_data=(X_val, Y_val) )

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))